In [ ]:
#talk about hw, https://www.quora.com/Why-is-AUC-Area-under-ROC-insensitive-to-class-distribution-changes 
#https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/
#ensemble,
#https://www.quora.com/What-will-happen-to-AUC-if-I-switch-the-positive-and-negative-classes-in-the-test-data
#https://medium.com/coinmonks/practical-tips-for-class-imbalance-in-binary-classification-6ee29bcdb8a7

In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, auc, roc_curve, f1_score,  confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score, roc_curve,roc_auc_score, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Import and store dataset
fraud_data = pd.read_csv('imbalancedFraudDF2.csv')
ipToCountry = pd.read_csv('IpAddress_to_Country.csv')

# from sklearn.grid_search import GridSearchCV is deprecated
import sklearn
print(sklearn.__version__)


0.20.2


In [2]:
#Distribution of the label column
fraud_data['class'].value_counts()

0    136961
1       849
Name: class, dtype: int64

In [3]:
fraud_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
3,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0
4,159135,2015-05-21 06:03:03,2015-07-09 08:05:14,42,ALEYXFXINSXLZ,Ads,Chrome,M,18,2.809315e+09,0


In [4]:
fraud_data.dtypes

user_id             int64
signup_time        object
purchase_time      object
purchase_value      int64
device_id          object
source             object
browser            object
sex                object
age                 int64
ip_address        float64
class               int64
dtype: object

In [5]:
#join country info based on ip_address
start = time.time()

countries = []
for i in range(len(fraud_data)):
    ip_address = fraud_data.loc[i, 'ip_address']
    tmp = ipToCountry[(ipToCountry['lower_bound_ip_address'] <= ip_address) &
                    (ipToCountry['upper_bound_ip_address'] >= ip_address)]
    if len(tmp) == 1:
        countries.append(tmp['country'].values[0])
    else:
        countries.append('NA')
        
fraud_data['country'] = countries
print "Lookup took", time.time() - start, "seconds."



Lookup took 196.983149052 seconds.


# Q: How to check if column user_id is unique(no dup)?


In [6]:
print fraud_data.user_id.nunique()#
print fraud_data.count()#
#most of the user_id has only the first 1 transaction, difficult to do time based aggregates, 
#e.g. amount in past 1 day for the user

137810
user_id           137810
signup_time       137810
purchase_time     137810
purchase_value    137810
device_id         137810
source            137810
browser           137810
sex               137810
age               137810
ip_address        137810
class             137810
country           137810
dtype: int64


In [7]:
fraud_data.to_csv("imbalancedFraudDFwithCountry2.csv", index=False)


In [4]:
fraud_data = pd.read_csv('imbalancedFraudDFwithCountry2.csv')

## Feature Engineering

In [8]:
#time related features: can be done before split, as they has no interaction between other rows, solely based on other columns of the same row
fraud_data['interval_after_signup'] = (pd.to_datetime(fraud_data['purchase_time']) - pd.to_datetime(
        fraud_data['signup_time'])).dt.total_seconds()

fraud_data['signup_days_of_year'] = pd.DatetimeIndex(fraud_data['signup_time']).dayofyear

fraud_data['signup_seconds_of_day'] = pd.DatetimeIndex(fraud_data['signup_time']).second + 60 * pd.DatetimeIndex(
    fraud_data['signup_time']).minute + 3600 * pd.DatetimeIndex(fraud_data['signup_time']).hour

fraud_data['purchase_days_of_year'] = pd.DatetimeIndex(fraud_data['purchase_time']).dayofyear
fraud_data['purchase_seconds_of_day'] = pd.DatetimeIndex(fraud_data['purchase_time']).second + 60 * pd.DatetimeIndex(
    fraud_data['purchase_time']).minute + 3600 * pd.DatetimeIndex(fraud_data['purchase_time']).hour

fraud_data = fraud_data.drop(['user_id','signup_time','purchase_time'], axis=1)




In [9]:
print fraud_data.source.value_counts()

SEO       55533
Ads       54701
Direct    27576
Name: source, dtype: int64


# Q: what now? train/test split? convert categorical features e.g. device_id to numericals? or smote?


In [ ]:
# What if convert source column by OHE now?
# it sneak peek the levels of this column in test data, what if there is another new level "newsPaper" in test, 
# but not in train?


In [11]:


y = fraud_data['class']
X = fraud_data.drop(['class'], axis=1)

#split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print("X_train.shape:", X_train.shape)
print("y_train.shape:", y_train.shape)


('X_train.shape:', (110248, 13))
('y_train.shape:', (110248,))


In [12]:
X_train.head()

,purchase_value,device_id,source,browser,sex,age,ip_address,country,interval_after_signup,signup_days_of_year,signup_seconds_of_day,purchase_days_of_year,purchase_seconds_of_day
117600,12,MLYMSJZROSFRX,SEO,Safari,F,20,8.336995e+08,Australia,706956.0,144,52524,152,68280
66167,72,HLUMNQTJFFJIU,Direct,Safari,M,32,2.546095e+09,United States,6133173.0,25,11171,96,9944
55238,30,TTBUPZUJXWVXC,Ads,Chrome,F,25,4.279183e+09,NA,7247527.0,127,35485,211,25412
38504,55,UPTZRZZQSDLSB,Ads,IE,M,32,3.628476e+09,United States,7869884.0,66,69719,157,77203
59263,20,ZBFDRBAHAAPZQ,SEO,Chrome,F,34,3.373047e+09,Brazil,4759614.0,35,41536,90,49150


# Convert categorical features to numericals



In [13]:
#converting needs to be done after split, and needs to be standalized/normalized, otherwise, 
#train count range for x: [1000, 5000] count, while test range [100, 500], different distribution :(

X_train = pd.get_dummies(X_train, columns=['source', 'browser'])#need to drop ['source', 'browser']? no, auto dropped by get_dummies 
X_train['sex'] = (X_train.sex == 'M').astype(int)

# the more a device is shared, the more suspicious
X_train['n_dev_shared'] = X_train.device_id.map(X_train.device_id.value_counts(dropna=False))#without dropna=False will produce nan in this col

# the more a ip is shared, the more suspicious
X_train['n_ip_shared'] = X_train.ip_address.map(X_train.ip_address.value_counts(dropna=False))

# the less visit from a country, the more suspicious
X_train['n_country_shared'] = X_train.country.map(X_train.country.value_counts(dropna=False))#lots of NAs in country column

X_train = X_train.drop(['device_id','ip_address','country'], axis=1)



##########################
X_test = pd.get_dummies(X_test, columns=['source', 'browser'])#need to drop ['source', 'browser']? no, auto dropped by get_dummies 
X_test['sex'] = (X_test.sex == 'M').astype(int)

# the more a device is shared, the more suspicious
X_test['n_dev_shared'] = X_test.device_id.map(X_test.device_id.value_counts(dropna=False))

# the more a ip is shared, the more suspicious
X_test['n_ip_shared'] = X_test.ip_address.map(X_test.ip_address.value_counts(dropna=False))

# the less visit from a country, the more suspicious
X_test['n_country_shared'] = X_test.country.map(X_test.country.value_counts(dropna=False))

X_test = X_test.drop(['device_id','ip_address','country'], axis=1)


# Q: What is the difference between normalization and standardization?


In [14]:


# normalize to [0,1], standardize to normal, mu=0, can < 0, so we do normalize here

min_max_scaler = preprocessing.MinMaxScaler()
#fit and transform the training data and use them for the model training
X_train[['n_dev_shared', 'n_ip_shared', 'n_country_shared']] = min_max_scaler.fit_transform(X_train[['n_dev_shared', 'n_ip_shared', 'n_country_shared']])


#after the model training on the transformed training data define the testing data df_test

#before the prediction of the test data, apply the same scaler obtained from above on X_test
X_test[['n_dev_shared', 'n_ip_shared', 'n_country_shared']] = min_max_scaler.fit_transform(X_test[['n_dev_shared', 'n_ip_shared', 'n_country_shared']])


/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [15]:
X_train.n_dev_shared.value_counts(dropna=False)

0.00    105485
0.25      4528
0.50       198
0.75        32
1.00         5
Name: n_dev_shared, dtype: int64

In [16]:
X_test.n_dev_shared.value_counts(dropna=False)

0.0    27235
0.5      324
1.0        3
Name: n_dev_shared, dtype: int64

# Simple RF model

In [19]:
classifier_RF = RandomForestClassifier(random_state=0)

classifier_RF.fit(X_train, y_train)

# predict class labels for the test set
predicted = classifier_RF.predict(X_test)

# generate class probabilities
probs = classifier_RF.predict_proba(X_test)


# generate evaluation metrics

print("%s: %r" % ("accuracy_score is: ", accuracy_score(y_test, predicted)))
print("%s: %r" % ("roc_auc_score is: ", roc_auc_score(y_test, probs[:, 1])))
print("%s: %r" % ("f1_score is: ", f1_score(y_test, predicted )))#string to int

print ("confusion_matrix is: ")
cm = confusion_matrix(y_test, predicted)
print cm
print 'recall =',float(cm[1,1])/(cm[1,0]+cm[1,1])
print 'precision =', float(cm[1,1])/(cm[1,1] + cm[0,1])#1.0

accuracy_score is: : 0.9962266889195269
roc_auc_score is: : 0.7371422642725146
f1_score is: : 0.6388888888888888
confusion_matrix is: 
[[27366     0]
 [  104    92]]
recall = 0.469387755102
precision = 1.0


In [30]:
classifier_RF

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=5, verbose=0, warm_start=False)

# SMOTE sampling

In [20]:

#https://beckernick.github.io/oversampling-modeling/

#Wrong way to sampling: smote before split train/test, then test data does not reflect true distribution in reality, 
#and “bleed” information from the test set into the training of the model. overfit! think about the case of simple 
#oversampling (where I just duplicate observations). If I upsample a dataset before splitting it into a train and 
#validation set, I could end up with the same observation in both datasets

#https://imbalanced-learn.org/en/stable/install.html

# Install
# imbalanced-learn is currently available on the PyPi’s reporitories and you can install it via pip:

# pip install -U imbalanced-learn

#oversampling on only the training data, the right way! 
#sampling_strategy = number of samples in the majority class over the number of samples in the minority class after resampling
sm = SMOTE(random_state=12)
x_train_sm, y_train_sm = sm.fit_sample(X_train, y_train)


unique, counts = np.unique(y_train_sm, return_counts=True)

print np.asarray((unique, counts)).T

[[     0 109595]
 [     1 109595]]


In [21]:
#RF on smoted training data
classifier_RF_sm = RandomForestClassifier(random_state=0)

classifier_RF_sm.fit(x_train_sm, y_train_sm)

# predict class labels for the test set
predicted_sm = classifier_RF_sm.predict(X_test)

# generate class probabilities
probs_sm = classifier_RF_sm.predict_proba(X_test)


# generate evaluation metrics

print("%s: %r" % ("accuracy_score_sm is: ", accuracy_score(y_test, predicted_sm)))
print("%s: %r" % ("roc_auc_score_sm is: ", roc_auc_score(y_test, probs_sm[:, 1])))
print("%s: %r" % ("f1_score_sm is: ", f1_score(y_test, predicted_sm )))#string to int

print ("confusion_matrix_sm is: ")
cm_sm = confusion_matrix(y_test, predicted_sm)
print cm_sm
print 'recall or sens_sm =',float(cm_sm[1,1])/(cm_sm[1,0]+cm_sm[1,1])
print 'precision_sm =', float(cm_sm[1,1])/(cm_sm[1,1] + cm_sm[0,1])



accuracy_score_sm is: : 0.9960089978956534
roc_auc_score_sm is: : 0.7367806692946857
f1_score_sm is: : 0.6099290780141844
confusion_matrix_sm is: 
[[27366     0]
 [  110    86]]
recall or sens_sm = 0.438775510204
precision_sm = 1.0


# Parameter tuning by GridSearchCV

In [22]:
parameters = {        
'max_depth': [None, 5, 15],
'n_estimators' :  [10,150],
'class_weight' : [{0: 1, 1: w} for w in [0.2, 1, 50, 100]]
}


scorers = {
    'precision_score': make_scorer(precision_score),
    'recall_score': make_scorer(recall_score),
    'f1_score': make_scorer(f1_score, pos_label=1)
}

clf = RandomForestClassifier(random_state=0)



In [23]:
def grid_search_wrapper(refit_score='f1_score'):
    """
    fits a GridSearchCV classifier using refit_score for optimization(refit on the best model according to refit_score)
    prints classifier performance metrics
    """
#     skf = StratifiedKFold(n_splits=10)
#     grid_search = GridSearchCV(clf, param_grid, scoring=scorers, refit=refit_score,
#                            cv=skf, return_train_score=True, n_jobs=-1)
    grid_search = GridSearchCV(clf, parameters, scoring=scorers, refit=refit_score,
                           cv=3, return_train_score=True, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # make the predictions
    y_pred = grid_search.predict(X_test)
    y_prob = grid_search.predict_proba(X_test)[:, 1]
    
    print('Best params for {}'.format(refit_score))
    print(grid_search.best_params_)

    # confusion matrix on the test data.
    print('\nConfusion matrix of Random Forest optimized for {} on the test data:'.format(refit_score))
    cm = confusion_matrix(y_test, y_pred)
    cmDF = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['0', '1'])
    print(cmDF)
    
    print("\t%s: %r" % ("roc_auc_score is: ", roc_auc_score(y_test, y_prob)))
    print("\t%s: %r" % ("f1_score is: ", f1_score(y_test, y_pred)))#string to int

    print 'recall = ', float(cm[1,1]) / (cm[1,0] + cm[1,1])
    print 'precision = ', float(cm[1,1]) / (cm[1, 1] + cm[0,1])

    return grid_search


## Optimizing on f1_score

In [24]:
grid_search_rf_f1 = grid_search_wrapper(refit_score='f1_score')


Best params for f1_score
{'n_estimators': 150, 'max_depth': None, 'class_weight': {0: 1, 1: 0.2}}

Confusion matrix of Random Forest optimized for f1_score on the test data:
   pred_0  pred_1
0   27366       0
1     104      92
	roc_auc_score is: : 0.7395749902679775
	f1_score is: : 0.6388888888888888
recall =  0.469387755102
precision =  1.0


In [25]:

best_rf_model_f1 = grid_search_rf_f1.best_estimator_
best_rf_model_f1

RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 0.2},
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=150, n_jobs=None, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [26]:
results_f1 = pd.DataFrame(grid_search_rf_f1.cv_results_)
results_sortf1 = results_f1.sort_values(by='mean_test_f1_score', ascending=False)
results_sortf1.head(100)

,mean_fit_time,mean_score_time,mean_test_f1_score,mean_test_precision_score,mean_test_recall_score,mean_train_f1_score,mean_train_precision_score,mean_train_recall_score,param_class_weight,param_max_depth,param_n_estimators,params,rank_test_f1_score,rank_test_precision_score,rank_test_recall_score,split0_test_f1_score,split0_test_precision_score,split0_test_recall_score,split0_train_f1_score,split0_train_precision_score,split0_train_recall_score,split1_test_f1_score,split1_test_precision_score,split1_test_recall_score,split1_train_f1_score,split1_train_precision_score,split1_train_recall_score,split2_test_f1_score,split2_test_precision_score,split2_test_recall_score,split2_train_f1_score,split2_train_precision_score,split2_train_recall_score,std_fit_time,std_score_time,std_test_f1_score,std_test_precision_score,std_test_recall_score,std_train_f1_score,std_train_precision_score,std_train_recall_score
12,1.664887,0.205714,0.684466,1.000000,0.520695,0.920571,1.000000,0.852972,"{0: 1, 1: 50}",None,10,"{u'n_estimators': 10, u'max_depth': None, u'cl...",1,1,3,0.702381,1.000000,0.541284,0.908407,1.000000,0.832184,0.654321,1.000000,0.486239,0.921933,1.000000,0.855172,0.696697,1.000000,0.534562,0.931373,1.000000,0.871560,0.045739,0.000405,0.021442,0.000000,0.024519,0.009425,0.000000,0.016150
9,12.893376,1.444693,0.684466,1.000000,0.520695,0.684713,1.000000,0.520679,"{0: 1, 1: 1}",5,150,"{u'n_estimators': 150, u'max_depth': 5, u'clas...",1,1,3,0.702381,1.000000,0.541284,0.675799,1.000000,0.510345,0.654321,1.000000,0.486239,0.699552,1.000000,0.537931,0.696697,1.000000,0.534562,0.678788,1.000000,0.513761,0.426533,0.006659,0.021442,0.000000,0.024519,0.010563,0.000000,0.012278
19,26.552913,2.577404,0.684466,1.000000,0.520695,1.000000,1.000000,1.000000,"{0: 1, 1: 100}",None,150,"{u'n_estimators': 150, u'max_depth': None, u'c...",1,1,3,0.702381,1.000000,0.541284,1.000000,1.000000,1.000000,0.654321,1.000000,0.486239,1.000000,1.000000,1.000000,0.696697,1.000000,0.534562,1.000000,1.000000,1.000000,0.383828,0.015907,0.021442,0.000000,0.024519,0.000000,0.000000,0.000000
17,27.086438,2.583402,0.684466,1.000000,0.520695,0.873096,1.000000,0.774887,"{0: 1, 1: 50}",15,150,"{u'n_estimators': 150, u'max_depth': 15, u'cla...",1,1,3,0.702381,1.000000,0.541284,0.862745,1.000000,0.758621,0.654321,1.000000,0.486239,0.884615,1.000000,0.793103,0.696697,1.000000,0.534562,0.871928,1.000000,0.772936,0.235094,0.030407,0.021442,0.000000,0.024519,0.008967,0.000000,0.014145
15,12.929599,1.464956,0.684466,1.000000,0.520695,0.684713,1.000000,0.520679,"{0: 1, 1: 50}",5,150,"{u'n_estimators': 150, u'max_depth': 5, u'clas...",1,1,3,0.702381,1.000000,0.541284,0.675799,1.000000,0.510345,0.654321,1.000000,0.486239,0.699552,1.000000,0.537931,0.696697,1.000000,0.534562,0.678788,1.000000,0.513761,0.378195,0.021576,0.021442,0.000000,0.024519,0.010563,0.000000,0.012278
13,26.368867,3.482888,0.684466,1.000000,0.520695,1.000000,1.000000,1.000000,"{0: 1, 1: 50}",None,150,"{u'n_estimators': 150, u'max_depth': None, u'c...",1,1,3,0.702381,1.000000,0.541284,1.000000,1.000000,1.000000,0.654321,1.000000,0.486239,1.000000,1.000000,1.000000,0.696697,1.000000,0.534562,1.000000,1.000000,1.000000,0.169863,0.187487,0.021442,0.000000,0.024519,0.000000,0.000000,0.000000
1,29.140872,2.888126,0.684466,1.000000,0.520695,1.000000,1.000000,1.000000,"{0: 1, 1: 0.2}",None,150,"{u'n_estimators': 150, u'max_depth': None, u'c...",1,1,3,0.702381,1.000000,0.541284,1.000000,1.000000,1.000000,0.654321,1.000000,0.486239,1.000000,1.000000,1.000000,0.696697,1.000000,0.534562,1.000000,1.000000,1.000000,0.768036,0.147296,0.021442,0.000000,0.024519,0.000000,0.000000,0.000000
11,27.412345,2.509964,0.684466,1.000000,0.520695,0.768321,1.000000,0.624054,"{0: 1, 1: 1}",15,150,"{u'n_estimators': 150, u'max_depth': 15, u'cla...",1,1,3,0.702381,1.000000,0.541284,0.757143,1.000000,0.609195,0.654321,1.000000,0.486239,0.789986,1.000000,0.652874,0.696697,1.000000,0.534562,0.757835,1.000000,0.610092,0.264141,0.010429,0.021442,0.

## Optimizing on recall_score

In [27]:
grid_search_rf_recall = grid_search_wrapper(refit_score='recall_score')
results_recall = pd.DataFrame(grid_search_rf_recall.cv_results_)
results_sortrecall = results_recall.sort_values(by='mean_test_recall_score', ascending=False)
# results_sortf1 = results_f1.sort_values(by='mean_test_f1_score', ascending=False)
results_sortrecall.head(100)
#recall is worse than default rf?? no this is on test, but train recall is better

Best params for recall_score
{'n_estimators': 150, 'max_depth': 5, 'class_weight': {0: 1, 1: 100}}

Confusion matrix of Random Forest optimized for recall_score on the test data:
   pred_0  pred_1
0   27362       4
1     104      92
	roc_auc_score is: : 0.7547512591969479
	f1_score is: : 0.6301369863013699
recall =  0.469387755102
precision =  0.958333333333


,mean_fit_time,mean_score_time,mean_test_f1_score,mean_test_precision_score,mean_test_recall_score,mean_train_f1_score,mean_train_precision_score,mean_train_recall_score,param_class_weight,param_max_depth,param_n_estimators,params,rank_test_f1_score,rank_test_precision_score,rank_test_recall_score,split0_test_f1_score,split0_test_precision_score,split0_test_recall_score,split0_train_f1_score,split0_train_precision_score,split0_train_recall_score,split1_test_f1_score,split1_test_precision_score,split1_test_recall_score,split1_train_f1_score,split1_train_precision_score,split1_train_recall_score,split2_test_f1_score,split2_test_precision_score,split2_test_recall_score,split2_train_f1_score,split2_train_precision_score,split2_train_recall_score,std_fit_time,std_score_time,std_test_f1_score,std_test_precision_score,std_test_recall_score,std_train_f1_score,std_train_precision_score,std_train_recall_score
21,12.363130,1.367976,0.594784,0.733601,0.526811,0.635602,0.751761,0.570452,"{0: 1, 1: 100}",5,150,"{u'n_estimators': 150, u'max_depth': 5, u'clas...",23,23,1,0.504132,0.458647,0.559633,0.546809,0.508911,0.590805,0.646341,0.963636,0.486239,0.714495,0.983871,0.560920,0.633880,0.778523,0.534562,0.645503,0.762500,0.559633,0.743508,0.188617,0.064303,0.208596,0.030461,0.068815,0.194050,0.014401
20,0.860347,0.147858,0.548968,0.610781,0.525289,0.580166,0.632198,0.561266,"{0: 1, 1: 100}",5,10,"{u'n_estimators': 10, u'max_depth': 5, u'class...",24,24,2,0.459459,0.396667,0.545872,0.470810,0.398724,0.574713,0.616715,0.829457,0.490826,0.675900,0.850174,0.560920,0.570732,0.606218,0.539171,0.593789,0.647696,0.548165,0.013349,0.007328,0.066018,0.176717,0.024523,0.084280,0.184629,0.010841
0,1.861617,0.202418,0.683771,0.997151,0.520695,0.922342,1.000000,0.856034,"{0: 1, 1: 0.2}",None,10,"{u'n_estimators': 10, u'max_depth': None, u'cl...",18,18,3,0.702381,1.000000,0.541284,0.909774,1.000000,0.834483,0.654321,1.000000,0.486239,0.923267,1.000000,0.857471,0.694611,0.991453,0.534562,0.933985,1.000000,0.876147,0.048920,0.005660,0.021065,0.004029,0.024519,0.009906,0.000000,0.017040
11,27.384468,2.484248,0.684466,1.000000,0.520695,0.768321,1.000000,0.624054,"{0: 1, 1: 1}",15,150,"{u'n_estimators': 150, u'max_depth': 15, u'cla...",1,1,3,0.702381,1.000000,0.541284,0.757143,1.000000,0.609195,0.654321,1.000000,0.486239,0.789986,1.000000,0.652874,0.696697,1.000000,0.534562,0.757835,1.000000,0.610092,0.299747,0.027647,0.021442,0.000000,0.024519,0.015322,0.000000,0.020382
19,25.341946,2.522901,0.684466,1.000000,0.520695,1.000000,1.000000,1.000000,"{0: 1, 1: 100}",None,150,"{u'n_estimators': 150, u'max_depth': None, u'c...",1,1,3,0.702381,1.000000,0.541284,1.000000,1.000000,1.000000,0.654321,1.000000,0.486239,1.000000,1.000000,1.000000,0.696697,1.000000,0.534562,1.000000,1.000000,1.000000,0.352971,0.022354,0.021442,0.000000,0.024519,0.000000,0.000000,0.000000
17,25.236453,2.424908,0.684466,1.000000,0.520695,0.873096,1.000000,0.774887,"{0: 1, 1: 50}",15,150,"{u'n_estimators': 150, u'max_depth': 15, u'cla...",1,1,3,0.702381,1.000000,0.541284,0.862745,1.000000,0.758621,0.654321,1.000000,0.486239,0.884615,1.000000,0.793103,0.696697,1.000000,0.534562,0.871928,1.000000,0.772936,0.378755,0.016360,0.021442,0.000000,0.024519,0.008967,0.000000,0.014145
15,12.148128,1.457909,0.684466,1.000000,0.520695,0.684713,1.000000,0.520679,"{0: 1, 1: 50}",5,150,"{u'n_estimators': 150, u'max_depth': 5, u'clas...",1,1,3,0.702381,1.000000,0.541284,0.675799,1.000000,0.510345,0.654321,1.000000,0.486239,0.699552,1.000000,0.537931,0.696697,1.000000,0.534562,0.678788,1.000000,0.513761,0.459535,0.026595,0.021442,0.000000,0.024519,0.010563,0.000000,0.012278
14,0.862300,0.149260,0.679670,0.979913,0.520695,0.682913,0.984742,0.522976,"{0: 1, 1: 50}",5,10,"{u'n_estimators': 10, u'max_depth': 5, u'class...",20,21,3,0.696165,0.975207,0.541284,0.675758,0.991111,0.512644,0.650307,0.981481,0.486239,0.692194,0.963115,0.540230,0.692537,0.983051,0.534562,0.680787,1.000000,0.516055,0.030047,0.004072,0.020816,

# Project tasks

In [96]:
#for task 3, based on the above var importance
trainDF = pd.concat([X_train, y_train], axis=1)
pd.crosstab(trainDF["n_dev_shared"],trainDF["class"])

class,0,1
n_dev_shared,,
0.0,104960,413
0.2,4397,359
0.4,167,208
0.6,35,97
0.8,9,31
1.0,4,20


In [77]:
fraudDF.groupby("class")[['interval_after_signup']].mean()


,interval_after_signup
class,
0,5.191179e+06
1,2.354059e+06


In [ ]:
fraudDF.groupby("class")[['interval_after_signup']].median()
#half of fraud happened 1 s after signed up

In [85]:
fraud_data[fraud_data['class'] == 1].head(100)

,user_id,signup_time,purchase_time,purchase_value,device_id,sex,age,ip_address,class,country,interval_after_signup,signup_days_of_year,signup_seconds_of_day,purchase_days_of_year,purchase_seconds_of_day,source_Ads,source_Direct,source_SEO,browser_Chrome,browser_FireFox,browser_IE,browser_Opera,browser_Safari
136961,86574,2015-01-11 14:39:04,2015-01-11 14:39:05,61,ZQVCVEXPSAZCA,1,23,3.941890e+08,1,United States,1.0,11,52744,11,52745,0,0,1,0,0,0,0,1
136962,74145,2015-03-27 07:55:56,2015-07-20 06:59:34,55,DFQZESNZDEIPP,0,18,1.930175e+09,1,Korea Republic of,9932618.0,86,28556,201,25174,1,0,0,0,1,0,0,0
136963,282058,2015-01-05 08:18:00,2015-01-05 08:18:01,36,EGLGSEGYPMMAM,1,37,3.503224e+09,1,United States,1.0,5,29880,5,29881,0,0,1,0,1,0,0,0
136964,255332,2015-01-01 01:46:58,2015-01-01 01:46:59,19,SQWWBFDRYPGYA,0,36,2.105295e+08,1,United States,1.0,1,6418,1,6419,0,0,1,0,0,1,0,0
136965,399497,2015-01-01 14:29:27,2015-01-22 15:14:12,52,JWAVUHXQXTCHW,1,22,1.052881e+09,1,Nigeria,1817085.0,1,52167,22,54852,0,1,0,1,0,0,0,0
136966,337297,2015-02-10 13:01:00,2015-04-14 05:08:57,50,BCKHXUUTHTMMW,0,31,8.000143e+08,1,Canada,5414877.0,41,46860,104,18537,0,0,1,0,1,0,0,0
136967,46163,2015-03-05 09:23:48,2015-06-03 01:57:00,12,YFFPJJLHEHZML,1,45,1.957369e+09,1,China,7749192.0,64,33828,154,7020,0,1,0,0,0,0,0,1
136968,17097,2015-01-08 10:34:16,2015-01-08 10:34:17,12,IGAXAVAZFJYOB,1,33,2.447058e+09,1,Netherlands,1.0,8,38056,8,38057,1,0,0,0,0,0,0,1
136969,318808,2015-01-02 06:01:09,2015-01-02 06:01:10,22,MLKPWLXKZYANO,0,33,2.543532e+09,1,United States,1.0,2,21669,2,21670,1,0,0,1,0,0,0,0
136970,303431,2015-01-11 19:16:05,2015-01-11 19:16:06,61,SIKJNYXDSEUEG,1,30,3.410478e+09,1,Taiwan; Republic of China (ROC),1.0,11,69365,11,69366,1,0,0,1,0,0,0,0


In [102]:
#for task 4, how to use the prediction:
t = (10 * probsBest_recall[:, 1]).astype(int)
unique, counts = np.unique(t, return_counts=True)

print np.asarray((unique, counts)).T
#green: 1 - 3 pass
#grey: 4-7 need manual investigation
#red: 8,9 decline

[[    1     2]
 [    2 26975]
 [    3   202]
 [    4    81]
 [    5   248]
 [    6     8]
 [    7     6]
 [    8    39]
 [    9   115]]


In [ ]:
#HW: try XGBOOST and compare the performance of RF